In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install ktrain
!pip3 install ktrain

In [3]:
# import ktrain
import ktrain
from ktrain import text
import pickle
import numpy as np

using Keras version: 2.2.4-tf


In [0]:
ktrain.__version__

'0.7.2'

In [0]:
DICT_PATH = "/content/drive/My Drive/Colab Notebooks/ML colab/dictionnaries/"
POS_NEG_WORDS_PATH = '/content/drive/My Drive/Colab Notebooks/ML colab/pos_neg_words/'
DATASETS_PATH = '/content/drive/My Drive/Colab Notebooks/ML colab/datasets/'
CLEANED_DATA_PATH = '/content/drive/My Drive/ML colab/cleaned_data/'
EMBEDDING_PATH = '/content/drive/My Drive/Colab Notebooks/ML colab/embedding/'

In [0]:
X_pos, X_neg,  X_test = load_cleaned_data_and_test('/content/drive/My Drive/ML colab/datasets/train_pos_full.txt', \
                   '/content/drive/My Drive/ML colab/datasets/train_neg_full.txt', '/content/drive/My Drive/ML colab/datasets/test_data.txt' ,HASHTAG = True, EMPHASIZE = True, PUNC=True, NUMBER = True, SMALL_WORDS = True , \
                        SLANG =True, APOSTROPHE = True, EMOJI = False, REPITITION = True, SPELLING = True, \
                        STOPWORDS = False, LEMMATIZE = False, STEMMING = False)
X = X_pos + X_neg
pickle.dump(X, open('/content/drive/My Drive/ML colab/cleaned_data/X_Cleaned_best1.pkl', 'wb'))
pickle.dump(X_test, open('/content/drive/My Drive/ML colab/cleaned_data/X_test_Cleaned_best1.pkl', 'wb'))

In [0]:
#load clean train and test data (10%) no repetition removal
X = pickle.load(open(CLEANED_DATA_PATH + 'sample_X_Cleaned.pkl', "rb"))
X_test = pickle.load(open(CLEANED_DATA_PATH + 'sample_X_test_Cleaned.pkl', "rb"))

In [0]:
#load clean train and test data FULL with repetition removal
X = pickle.load(open(CLEANED_DATA_PATH + 'sample_X_Cleaned_lstm.pkl', "rb"))
X_test = pickle.load(open(CLEANED_DATA_PATH + 'sample_X_test_Cleaned_lstm.pkl', "rb"))

In [0]:
y = np.array(100000 * [0] + 100000 * [1]) # on purpose in order to use relu activation function

In [0]:
y = np.array(1250000 * [0] + 1250000 * [1]) # on purpose in order to use relu activation function

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.05)

In [13]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=[0,1],
                                                                       preprocess_mode='bert',
                                                                       maxlen=30, 
                                                                       max_features=100000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [14]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 30
done.


In [0]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 2375000 samples, validate on 125000 samples
1247382/2375000 [==============>...............] - ETA: 3:39:43 - loss: 0.3320 - acc: 0.8517

In [0]:
X_kaggle = pickle.load(open(CLEANED_DATA_PATH + 'sample_X_test_Cleaned_lstm.pkl', "rb"))

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [0]:
pred = predictor.predict(X_kaggle)

In [0]:
new_pred = [1 if x == 0 else -1 for x in pred]

In [0]:
import csv
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [0]:
  ids = np.arange(1,10001)
  name2 = '/content/drive/My Drive/ML colab/submission_BertModel3.csv'
  create_csv_submission(ids,new_pred,name2)

In [0]:
predictor.save('/content/drive/My Drive/ML colab/Bert Models/BertModel3')

In [0]:
hidhfg, dpg,dfgff ohfnhofijgi,